In [1]:
%load_ext autoreload
%autoreload 2
CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())

PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

## The Data

In [2]:
import dataset
from dataset import AudioDataset

# probably want to collect some RAM before we go.
import gc
gc.collect()

train_data = AudioDataset('train-clean-100')
val_data = AudioDataset('dev-clean')


/home/alice/anaconda3/envs/lowergcc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


create dataset from data  ../../data/train-clean-100
	total mfcc cnt:  28539
	total transcript cnt:  28539
create dataset from data  ../../data/dev-clean
	total mfcc cnt:  2703
	total transcript cnt:  2703


In [22]:
mfcc,  transcript = val_data[0]
print(mfcc.shape, mfcc.dtype)
print(transcript)

mfcc,  transcript = val_data[1]
print(mfcc.shape, mfcc.dtype)
print(transcript)

(830, 28) float32
[63 65 78 68 32 84 72 69 32 80 79 79 82 32 83 73 76 76 89 32 84 72 73 78
 71 83 32 82 85 70 70 76 69 68 32 85 80 32 84 72 69 73 82 32 70 69 65 84
 72 69 82 83 32 65 78 68 32 76 79 79 75 69 68 32 77 73 83 69 82 65 66 76
 69 32 65 83 32 79 78 76 89 32 65 32 76 73 84 84 76 69 32 66 73 82 68 32
 67 65 78 32 76 79 79 75 32 87 72 69 78 32 73 84 32 73 83 32 85 78 72 65
 80 80 89 64]
(684, 28) float32
[63 72 69 32 84 72 79 85 71 72 84 32 73 84 32 87 65 83 32 65 32 76 65 83
 84 32 66 85 82 83 84 32 79 70 32 69 78 69 82 71 89 32 72 69 32 75 78 69
 87 32 72 79 87 32 67 76 79 83 69 32 84 72 69 89 32 66 79 84 72 32 87 69
 82 69 32 84 79 32 69 88 72 65 85 83 84 73 79 78 64]


In [3]:
from torch.utils.data import DataLoader

batch_size = 64
train_loader =  DataLoader(
            train_data,
            batch_size=batch_size,
            drop_last=True,
            shuffle=True,
            collate_fn=train_data.collate_fn
)


print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))

# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break


Train dataset samples = 28539, batches = 445
torch.Size([64, 1683, 28]) torch.Size([64, 287]) torch.Size([64]) torch.Size([64])


In [7]:
from model import ASRModel
from defines import PHONEMES

model = ASRModel(
    input_size = 28, 
    embed_size= 64,
    output_size = len(PHONEMES)
)
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    model.forward(x, lx)
    break

torch.Size([64, 1671, 28]) torch.Size([64, 269]) torch.Size([64]) torch.Size([64])


AttributeError: 'ASRModel' object has no attribute 'decoder'

## The Model